# The Deep Learning Project

Team members:
- Mellissa HAFIS
- Ilona LEDROGOFF

In [ ]:
!wget https://www.lamsade.dauphine.fr/~cazenave/project2026.zip
!unzip project2026.zip
!ls -ly

--2026-01-19 07:09:12--  https://www.lamsade.dauphine.fr/~cazenave/project2026.zip
Resolving www.lamsade.dauphine.fr (www.lamsade.dauphine.fr)... 193.48.71.250
Connecting to www.lamsade.dauphine.fr (www.lamsade.dauphine.fr)|193.48.71.250|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 138578548 (132M) [application/zip]
Saving to: ‘project2026.zip’

project2026.zip     100%[===================>] 132.16M  23.1MB/s    in 7.5s    

2026-01-19 07:09:20 (17.6 MB/s) - ‘project2026.zip’ saved [138578548/138578548]

Archive:  project2026.zip
  inflating: games.data              
  inflating: golois.cpython-312-x86_64-linux-gnu.so  
ls: invalid option -- 'y'
Try 'ls --help' for more information.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


# Version 2

In [ ]:
!pip install tensorflow

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras import regularizers
import gc

import golois

planes = 31
moves = 361
N = 50000
epochs = 20
batch = 128
filters = 32

input_data = np.random.randint(2, size=(N, 19, 19, planes))
input_data = input_data.astype ('float32')

In [ ]:
policy = np.random.randint(moves, size=(N,))
policy = keras.utils.to_categorical (policy)
policy = policy.astype ('float32')

value = np.random.randint(2, size=(N,))
value = value.astype ('float32')

end = np.random.randint(2, size=(N, 19, 19, 2))
end = end.astype ('float32')

groups = np.zeros((N, 19, 19, 1))
groups = groups.astype ('float32')

print ("Tensorflow version", tf.__version__)
print ("getValidation", flush = True)
golois.getValidation (input_data, policy, value, end)

Tensorflow version 2.19.0
getValidation


In [ ]:
epochs = 500

In [ ]:
input_data.shape

(50000, 19, 19, 31)

## Some improvements:
- **Version 1:**
### In the feature extractor:
- We change the simple Conv2D into residual blocks
- It's supposed to help the network to reuse the features efficiently and prevent gradient vanishing problem.

### Adding a dropout in the value head
- To prevent overfitting

### Increasing the number of filters
- possible values: 35, 40, 45, 48, ...
- MAX PARMS: 100 000

### Ajouter une mini-conv avant les têtes
---

- **Version 2:**
- Reduce the number of the parameters
- Reducing the number of residual blocks (e.g., from 5 to 4).
- Adding batch normalization to stabilize deeper layers.
  - After each convolution
  - Before activation (ReLU)
- Fine-tuning the dropout rate and L2 regularization for better generalization.


In [ ]:
# Define checkpoint directory (persistent on Drive)
import os
import json
checkpoint_dir = "/content/drive/MyDrive/Master-2-IASD-app/Deep-learning/ResNet/checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)

checkpoint_path = os.path.join(checkpoint_dir, "model_last.keras")
progress_file = os.path.join(checkpoint_dir, "progress.json")


In [ ]:
progress_file = "/content/drive/MyDrive/Master-2-IASD-app/Deep-learning/ResNet/checkpoints/progress.json"
if os.path.exists(progress_file):
    !cat {progress_file}
else:
    print(f"File not found: {progress_file}")

{"last_epoch": 765}

In [ ]:
def batchNorm_residual_block(x, filters):
    shortcut = x

    # First conv + BN + ReLU
    x = layers.Conv2D(filters, 3, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    # Second conv + BN (no ReLU yet)
    x = layers.Conv2D(filters, 3, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)

    # Add shortcut connection
    x = layers.Add()([x, shortcut])

    # Final ReLU after addition
    x = layers.Activation('relu')(x)

    return x


# features extractor

input = keras.Input(shape=(19, 19, planes), name='board')

x = layers.Conv2D(filters, 1, padding='same', use_bias=False)(input)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)

for i in range(4):
    x = batchNorm_residual_block(x, filters)


# policy head
policy_head = layers.Conv2D(2, 1, activation='relu', padding='same')(x)
policy_head = layers.Conv2D(1, 1, activation='relu', padding='same', use_bias = False, kernel_regularizer=regularizers.l2(0.0001))(policy_head)
policy_head = layers.Flatten()(policy_head)
policy_head = layers.Activation('softmax', name='policy')(policy_head)

# value head
value_head = layers.Conv2D(2, 1, activation='relu', padding='same')(x)
value_head = layers.Conv2D(1, 1, activation='relu', padding='same', use_bias = False, kernel_regularizer=regularizers.l2(0.0001))(value_head)
value_head = layers.Flatten()(value_head)
value_head = layers.Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.001))(value_head)
value_head = layers.Dropout(0.3)(value_head)
value_head = layers.Dense(1, activation='sigmoid', name='value', kernel_regularizer=regularizers.l2(0.0001))(value_head)

model = keras.Model(inputs=input, outputs=[policy_head, value_head])

model.summary ()

# compilation
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.00099), # init-> 0.0001
              loss={'policy': 'categorical_crossentropy', 'value': 'mse'},
              loss_weights={'policy' : 1.0, 'value' : 1.0},
              metrics={'policy': 'categorical_accuracy', 'value': 'mae'})


# resume if previous checkpoint exists
start_epoch = 0
if os.path.exists(checkpoint_path) and os.path.exists(progress_file):
    print("Loading model from checkpoint...")
    model = keras.models.load_model(checkpoint_path)
    with open(progress_file, "r") as f:
        start_epoch = json.load(f)["last_epoch"]
    print(f"Resuming from epoch {start_epoch + 1}")
else:
    print("Starting training from scratch")

# training
for i in range(start_epoch + 1, start_epoch + epochs + 1):
    print('epoch ' + str (i))
    golois.getBatch (input_data, policy, value, end, groups, i * N)
    history = model.fit(input_data,
                        [policy,value],
                        epochs=1, batch_size=batch)
    # save JSON progress every epoch (lightweight)
    with open(progress_file, "w") as f:
        json.dump({"last_epoch": i}, f)

    if (i % 5 == 0):
        gc.collect ()
    # Save progress every 20 epochs
    if (i % 20 == 0):
        golois.getValidation (input_data, policy, value, end)
        val = model.evaluate (input_data,
                              [policy, value], verbose = 0, batch_size=batch)
        print("val =", val)
        # save model + progress
        model.save(checkpoint_path)
        print(f"Saved checkpoint at epoch {i}")

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ board (InputLayer)  │ (None, 19, 19,    │          0 │ -                 │
│                     │ 31)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 19, 19,    │        992 │ board[0][0]       │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 19, 19,    │        128 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 19, 19,    │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 19, 19,    │      9,216 │ activation[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 19, 19,    │        128 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 19, 19,    │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 19, 19,    │      9,216 │ activation_1[0][… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 19, 19,    │        128 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 19, 19,    │          0 │ batch_normalizat… │
│                     │ 32)               │            │ activation[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 19, 19,    │          0 │ add[0][0]         │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 19, 19,    │      9,216 │ activation_2[0][… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 19, 19,    │        128 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 19, 19,    │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 19, 19,    │      9,216 │ activation_3[0][… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 19, 19,    │        128 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 19, 19,    │          0 │ batch_normalizat

 Total params: 94,159 (367.81 KB)

 Trainable params: 93,583 (365.56 KB)

 Non-trainable params: 576 (2.25 KB)

Loading model from checkpoint...
Resuming from epoch 506
epoch 506
391/391 ━━━━━━━━━━━━━━━━━━━━ 20s 35ms/step - loss: 2.4326 - policy_categorical_accuracy: 0.4032 - policy_loss: 2.3125 - value_loss: 0.1201 - value_mae: 0.2926
epoch 507
391/391 ━━━━━━━━━━━━━━━━━━━━ 14s 35ms/step - loss: 2.4163 - policy_categorical_accuracy: 0.4068 - policy_loss: 2.2963 - value_loss: 0.1200 - value_mae: 0.2922
epoch 508
391/391 ━━━━━━━━━━━━━━━━━━━━ 14s 35ms/step - loss: 2.4257 - policy_categorical_accuracy: 0.4020 - policy_loss: 2.3046 - value_loss: 0.1211 - value_mae: 0.2941
epoch 509
391/391 ━━━━━━━━━━━━━━━━━━━━ 13s 34ms/step - loss: 2.4064 - policy_categorical_accuracy: 0.4065 - policy_loss: 2.2870 - value_loss: 0.1194 - value_mae: 0.2913
epoch 510
391/391 ━━━━━━━━━━━━━━━━━━━━ 13s 34ms/step - loss: 2.4278 - policy_categorical_accuracy: 0.4057 - policy_loss: 2.3070 - value_loss: 0.1208 - value_mae: 0.2936
epoch 511
391/391 ━━━━━━━━━━━━━━━━━━━━ 13s 34ms/step - loss: 2.4176 - policy_categorical_accuracy:

In [ ]:
model = keras.models.load_model(checkpoint_path)
print(f"Model loaded from {checkpoint_path}")

Model loaded from /content/drive/MyDrive/Master-2-IASD-app/Deep-learning/ResNet/checkpoints/model_last.keras


In [ ]:
# Save the model to Google Drive
model_save_path = "/content/drive/MyDrive/Master-2-IASD-app/Deep-learning/ResNet/checkpoints/Mellissa_HAFIS-Ilona_LEDROGOFF-ResNet.h5"
model.save(model_save_path)
print(f"Model saved to {model_save_path}")

Model saved to /content/drive/MyDrive/Master-2-IASD-app/Deep-learning/ResNet/checkpoints/Mellissa_HAFIS-Ilona_LEDROGOFF-ResNet.h5


In [ ]:
model.save('Mellissa_HAFIS-Ilona_LEDROGOFF-v.h5')

Model saved as Mellissa_HAFIS-Ilona_LEDROGOFF-v12.h5


v3(40 epochs): gave same results approximatly 0.37

v4 60 epochs:
val = [2.61161470413208, 2.491255044937134, 0.12023338675498962, 0.3767400085926056, 0.29251644015312195]


v5 80 epochs:

val = [2.5814108848571777, 2.4611072540283203, 0.12023502588272095, 0.3830600082874298, 0.29250794649124146]


v6 100 epochs:
val = [2.563153028488159, 2.4429502487182617, 0.12023553997278214, 0.3858200013637543, 0.2925105392932892]

v7 120 epochs
:val = [2.5540621280670166, 2.433702230453491, 0.12023333460092545, 0.3862200081348419, 0.2925240397453308]


v8 140 : val = [2.532742738723755, 2.4124836921691895, 0.12023456394672394, 0.38901999592781067, 0.2925328314304352]

v9: 160 (learning rate passe a 0,0006):
val = [2.5364038944244385, 2.4161550998687744, 0.1202331930398941, 0.38934001326560974, 0.292520672082901]


v10 180 (learning rate passe a 0,0007):
val = [2.5080697536468506, 2.3878350257873535, 0.12024098634719849, 0.39430001378059387, 0.2925296127796173]

v11 200 epochs ; (learning rate passe a 0,0009)
val = [2.4943742752075195, 2.374094247817993, 0.12023616582155228, 0.39452001452445984, 0.29251357913017273]

v12 220 epochs ( 0.0009):
val = [2.5123374462127686, 2.3920929431915283, 0.12023553997278214, 0.3924199938774109, 0.2925108075141907]

v13 Saved checkpoint at epoch 440  ( learning rate 0.0009).
val = [2.449185609817505, 2.3289377689361572, 0.1202370747923851, 0.4026600122451782, 0.2925172746181488]


v14: Saved checkpoint at epoch 460 ( learning rate 0.0009).
val = [2.4581916332244873, 2.337979555130005, 0.12023373693227768, 0.39937999844551086, 0.29251253604888916]

val = [2.438472032546997, 2.3182597160339355, 0.12023499608039856, 0.40345999598503113, 0.29250776767730713]
Saved checkpoint at epoch 520

val = [2.4382171630859375, 2.318016529083252, 0.12024436891078949, 0.4046599864959717, 0.2925556004047394]
Saved checkpoint at epoch 540

val = [2.433934450149536, 2.313666582107544, 0.12023387849330902, 0.405239999294281, 0.2925293743610382]
Saved checkpoint at epoch 560

val = [2.4352598190307617, 2.315086841583252, 0.12023459374904633, 0.4044800102710724, 0.2925083339214325]
Saved checkpoint at epoch 580

val = [2.4337258338928223, 2.3134713172912598, 0.12024436146020889, 0.40303999185562134, 0.2925376892089844]
Saved checkpoint at epoch 600

val = [2.4322550296783447, 2.3120310306549072, 0.12023569643497467, 0.4032599925994873, 0.29251131415367126]
Saved checkpoint at epoch 620

val = [2.429220676422119, 2.3090097904205322, 0.12023370712995529, 0.4042600095272064, 0.2925277352333069]
Saved checkpoint at epoch 640

val = [2.432229518890381, 2.3119966983795166, 0.1202368512749672, 0.40478000044822693, 0.29251644015312195]
Saved checkpoint at epoch 660

val = [2.4379899501800537, 2.31775164604187, 0.12023387104272842, 0.4048199951648712, 0.292511910200119]
Saved checkpoint at epoch 680

val = [2.421626329421997, 2.301339626312256, 0.12023693323135376, 0.4053399860858917, 0.2925170063972473]
Saved checkpoint at epoch 700

val = [2.419804096221924, 2.2995803356170654, 0.12023426592350006, 0.4069199860095978, 0.2925100028514862]
Saved checkpoint at epoch 720

val = [2.446979284286499, 2.326749324798584, 0.12023457139730453, 0.4032999873161316, 0.2925080955028534]
Saved checkpoint at epoch 740

val = [2.4236278533935547, 2.303400754928589, 0.12023551762104034, 0.4061799943447113, 0.2925104796886444]
Saved checkpoint at epoch 760


